<a href="https://colab.research.google.com/github/sairamadithya/OCT-classification/blob/main/OCT_images_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from keras import Sequential
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

The dataset is organized into 3 folders (train, test, val) and contains subfolders for each image category (NORMAL,CNV,DME,DRUSEN). There are around 40k X-Ray images (JPEG) and 4 categories (NORMAL,CNV,DME,DRUSEN). 

This is a subset of the original dataset. More the training images, more the training time and we would require higher configurations of GPU. The common error associated with the training is OOM (Out Of Memory)

Images are labeled as (disease)-(randomized patient ID)-(image number by this patient) and split into 4 directories: CNV, DME, DRUSEN, and NORMAL.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!jar xvf '/content/drive/MyDrive/OCT images.zip'

We are using imagedatagenerator to generate samples from the given image dataset by applying the mentioned transforms.

In [ ]:
train_dir= image.ImageDataGenerator(height_shift_range=0.1,width_shift_range=0.1,horizontal_flip=True)
test_dir= image.ImageDataGenerator(height_shift_range=0.1,width_shift_range=0.1,horizontal_flip=True)

In [ ]:
train_set= train_dir.flow_from_directory(
    '/content/archive (1)/OCT2017/train',
    batch_size=50,
    class_mode='categorical',
    target_size=(224,224)
)
test_set= test_dir.flow_from_directory(
    '/content/archive (1)/OCT2017/test',
    shuffle=False,
    class_mode='categorical',
    target_size=(224,224)
)

Found 40000 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [ ]:
train_set.class_indices

{'CNV': 0, 'DME': 1, 'DRUSEN': 2, 'NORMAL': 3}

I have developed a transfer learning based MobileNetV2 model. I have added certain additional layers to overcome the problem of overfitting.

GaussianNoise- this is a type of noise added to the data so as to increase the robustness and to introduce augmentation.

dropout- this is used to reduce the model complexity by randomly cutting off certain nerual connections.

In [ ]:
base_model = tensorflow.keras.applications.MobileNetV2(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base_model.layers:
    layer.trainable = True
model=Sequential()
model.add(base_model)
model.add(GaussianNoise(0.25))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(GaussianNoise(0.25))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(4,activation='softmax'))
model.summary()


9420800/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 gaussian_noise (GaussianNoi  (None, 7, 7, 1280)       0         
 se)                                                             
                                                                 
 batch_normalization (BatchN  (None, 7, 7, 1280)       5120      
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 64)                4014

This model is compiled using adagrad optimizer, crossentropy loss function and metrics i have used is accuracy. You can add AUC, precision, recall and other classification metrics.

In [ ]:
model.compile(optimizer='AdaGrad',loss='categorical_crossentropy',metrics=['accuracy'])

This is a very good dataset and training for less epochs gives good results. I trained the model for 10 epochs with 80 steps per each epoch.

In [ ]:
from keras.callbacks import EarlyStopping
es=EarlyStopping(monitor='val_loss',patience=5)
history=model.fit(
    train_set,
    epochs=10,
    steps_per_epoch= 80,
    validation_data= test_set,
    callbacks=es
)

Epoch 1/10
80/80 [==============================] - 89s 1s/step - loss: 0.9167 - accuracy: 0.6425 - val_loss: 1.2723 - val_accuracy: 0.5258
Epoch 2/10
80/80 [==============================] - 84s 1s/step - loss: 0.5887 - accuracy: 0.7845 - val_loss: 1.2294 - val_accuracy: 0.5671
Epoch 3/10
80/80 [==============================] - 82s 1s/step - loss: 0.4795 - accuracy: 0.8342 - val_loss: 0.9612 - val_accuracy: 0.6374
Epoch 4/10
80/80 [==============================] - 83s 1s/step - loss: 0.4250 - accuracy: 0.8580 - val_loss: 0.6209 - val_accuracy: 0.7696
Epoch 5/10
80/80 [==============================] - 83s 1s/step - loss: 0.3903 - accuracy: 0.8683 - val_loss: 0.4146 - val_accuracy: 0.8481
Epoch 6/10
80/80 [==============================] - 82s 1s/step - loss: 0.3594 - accuracy: 0.8780 - val_loss: 0.3269 - val_accuracy: 0.8781
Epoch 7/10
80/80 [==============================] - 82s 1s/step - loss: 0.3493 - accuracy: 0.8892 - val_loss: 0.3598 - val_accuracy: 0.8781
Epoch 8/10
80/80 [==

The model achieved the following results:-
training accuracy- 78%

training loss- 0.598

validation accuracy- 94.11%

validation loss- 0.186

In [ ]:
model.evaluate(train_set)

800/800 [==============================] - 556s 695ms/step - loss: 0.5985 - accuracy: 0.7876


[0.5985212922096252, 0.7876250147819519]

In [ ]:
model.evaluate(test_set)

31/31 [==============================] - 14s 440ms/step - loss: 0.1868 - accuracy: 0.9411


[0.18679122626781464, 0.94111567735672]

In [ ]:
tf.keras.models.save_model(model,'mymodel.hdf5')

I have developed a website that asks the OCT image from the user in the form of JPG, PNG or JPEG. Once that is uploaded, it will classify into what the eye disease is present in that OCT image.

In [ ]:
!pip install streamlit

     |████████████████████████████████| 9.9 MB 5.3 MB/s 
     |████████████████████████████████| 4.3 MB 30.5 MB/s 
     |████████████████████████████████| 181 kB 44.4 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 111 kB 47.0 MB/s 
     |████████████████████████████████| 164 kB 45.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 128 kB 46.3 MB/s 
     |████████████████████████████████| 793 kB 41.0 MB/s 
     |████████████████████████████████| 380 kB 42.8 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=829ac1724a1c1b9eabb11487c7e154ce30ec964db9bf81bf7a913dece98d9c10
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      S

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import streamlit as st


@st.cache(allow_output_mutation=True)
def load_model():
  model=tf.keras.models.load_model('/content/mymodel.hdf5')
  return model
with st.spinner('Model is being loaded..'):
  model=load_model()

st.write("""
         # OCT IMAGE DIAGNOSIS USING DEEP LEARNING
           by V A SAIRAM 
         """
)
file = st.file_uploader("Please upload any image from the local machine in case of computer or upload camera image in case of mobile", type=["jpg", "png","jpeg","webp"])
import cv2
from PIL import Image, ImageOps
import numpy as np
st.set_option('deprecation.showfileUploaderEncoding', False)
def import_and_predict(image_data, model):
    
        size = (224,224)    
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_reshape = img[np.newaxis,...]
    
        prediction = model.predict(img_reshape)
        return prediction
if file is None:
    st.text("Please upload an image file within the allotted file size or retry of the error still persists...")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = import_and_predict(image, model)
    class_names= ['CNV', 
                  'DME',
                  'DRUSEN',
                  'NORMAL'
                  ]
    string= "THE PARTICULAR WASTE CATEGORY IS: " + class_names[np.argmax(predictions)]
    st.success(string) 
    #string1="Most of the wastes cannot be disposed at our homes. There are special collection sites for the disposal. Our work is to isolate the wastes into seperate containers which can be send to the collection sites for proper disposal."  
    string2="HOPE THIS HELPS!! THANK YOU FOR USING THIS PROJECT!!!"
    st.success(string2)
  

Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-03-11 06:19:57.098 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.225.194.61:8501

npx: installed 22 in 5.756s
your url is: https://massive-bear-38.loca.lt
  Stopping...
^C
